<div style="color:red;background-color:black">
Diamond Light Source
<br style="color:red;background-color:antiquewhite"><h1>C Programming: Shared Libraries</h1><br>
©2000-21 Chris Seddon 
</div>

# 1: Build the library (linker fails)
Shared libraries are loaded into virtual memory by executable programs when they start. This enables many executables to share library code at run time.  This means the shared library code does not need to be linked to the excutable (unlike with static libraries) thereby greatly reducing the size of the executable.  

We begin by compiling and linking the shared library code using the <b>-shared</b><a href="http://www.microhowto.info/howto/build_a_shared_library_using_gcc.html#idp25488"><b>-fPIC</b></a> options of gcc.  

However, when the executable is linked, the linker will not know where the shared library is located.

In [1]:
cd mylib
gcc -shared -fPIC -o libf.so f1.c f2.c f3.c

cd ../src
gcc f_main.c -o f.exe
cd ..

/tmp/ccAXx4XG.o: In function `main':
f_main.c:(.text+0xa): undefined reference to `f1'
f_main.c:(.text+0x14): undefined reference to `f2'
f_main.c:(.text+0x1e): undefined reference to `f3'
collect2: error: ld returned 1 exit status


## 2: Place the shared library in the default location (/usr/lib)

We can locate our library with other system libraries by copying it to `/usr/lib`.  Note, however, you will need `sudo` privileges to do that.  The kernel needs to be informed of the location of the new shared library, so the kernel cache needs updating.

Unfortunately, the Jupyter bash kernel we are using does support reading on stdin, so we can't use `sudo` in our notebook.  

Instead you will need to run the following on the command line:

In [ ]:
# cd mylib
# sudo cp libf.so /usr/lib

# update cache
# sudo ldconfig
# cd ..

Once the library is installed in `/usr/lib` we can compile and link our main program and then run it:

In [ ]:
cd src
gcc f_main.c -lf -o f.exe
f.exe
cd ..

Using `sudo` is problematic for ordinary users.  But, before we look at alternatives, let's remove the shared library from `/usr/lib`.  Again, we will need to do the following on the command line, outside the notebook:

In [ ]:
# sudo rm /usr/lib/libf.so

## 3: Using LD_LIBRARY_PATH (works but fragile)
If you can't use sudo, then one alternative is to use the `LD_LIBRARY_PATH` environment variable to locate the shared library.
This method works, but is fragile (the environment variable must be set correctly).  

We begin by compiling the library code:

In [ ]:
cd mylib
gcc -shared -fPIC -o libf.so f1.c f2.c f3.c
cd ..

Next we setup the `LD_LIBRARY_PATH` relative to directory in which we compile our main program.  Then we can compile and run our executable:

In [ ]:
cd src
export LD_LIBRARY_PATH=../mylib
gcc f_main.c -L../mylib -lf -o f.exe

# run the program
f.exe

cd ..

We can check where our shared library resides, by using `ldd`:

In [ ]:
cd src

# check shared library dependencies
ldd f.exe
cd ..

## 4: Build Shared Library with RPATH (most reliable)
Rather than using `LD_LIBRAY_PATH` to locate the library, we can embed the path to the library in the executable as a relative path.  This involves talking to the linker directly.  Recall `gcc` is a macro that calls the compiler and the linker.  

To pass parameters to the linker we use the `-Wl,` option to `gcc`.

In [ ]:
cd src
gcc f_main.c -Wl,-rpath=../mylib -L../mylib -lf -o f.exe
f.exe
ldd f.exe
cd ..

Finally, let's look at info stored in the shared object.  Firstly use the `file` command:

In [ ]:
cd mylib
file libf1.so
cd ..

Secondly, use the `nm` command.  Note that we have not compiled with `-g` so debugging symbols will be missing:

In [ ]:
cd mylib
nm libf1.so
cd ..